当普通用户通过浏览器打开该网页时，需要输入邮箱和密码，然后单击登录
按钮将数据提交到服务端。如果登录成功，则会跳转到主页；否则，会跳转回登录
页。下面是尝试自动化处理该流程的初始版本代码

In [ ]:
from urllib.parse import urlencode
from urllib.request import Request, urlopen
LOGIN_URL = 'http://example.python-scraping.com/user/login'
LOGIN_EMAIL = 'example@python-scraping.com'
LOGIN_PASSWORD = 'example'
data = {'email': LOGIN_EMAIL, 'password': LOGIN_PASSWORD}
encoded_data = urlencode(data)
request = Request(LOGIN_URL, encoded_data.encode('utf-8'))
response = urlopen(request)
print(response.geturl())

我们可以使用requests以几行代码实现同样的处理。

In [8]:
import requests
response = requests.post(LOGIN_URL, data)
print(response.url)

http://example.python-scraping.com/user/login


In [9]:
from chp6.login import parse_form
html = requests.get(LOGIN_URL)
form = parse_form(html.content)
print(form)


{'email': '', 'password': '', 'remember_me': 'on', '_next': '/places/default/index', '_formkey': '10b706d3-7b61-4231-b17a-5429201fa4c9', '_formname': 'login'}


In [10]:
html = requests.get(LOGIN_URL)
data = parse_form(html.content)
data['email'] = LOGIN_EMAIL
data['password'] = LOGIN_PASSWORD
response = requests.post(LOGIN_URL, data)
response.url


'http://example.python-scraping.com/user/login'

In [11]:
response.cookies.keys()

['session_data_places', 'session_id_places']

In [12]:
response.cookies.values()

['"hmac256:48ece30389623b366345457efd119db3e00a2d683babf1d094a84613f18b62d4:KA8afZO9CAQekQbCCw_jxjRjhjpB3saHKpRK98ryBg0_QtuRpfz5m1I64OBxXqbsy_xRmzOpNY9-LTCXjz6zYpcQ3cWYFVPC7_uVF8LHE423XpLrEeFLZkIQyd0FkldU1davhHhUKsfmL0oFK3cqBZRcyPan4YRR_YFiyEdi-L9GOSwihDGmD42j6R3AjTna-zRWgLxLHQrBPYEXfanYODmupB_bexLsbCE8RJBaCFw25mcMpWGxtg_2uzUPU6ejnIkNOfoGjeJQhFgZgWgjPbwdzhoCuAj1MNbrLCckFi0qJmyhUVUTJFbsS3ZoLG9VVeLPJr1xF8YrLm68W6BnfCkviX8Atf8XmkCjlgXdNRupIMzNadt5jR6e3Wusfl1y"',
 'True']

In [13]:
second_response = requests.post(LOGIN_URL, data, cookies=html.cookies)
second_response.url

'http://example.python-scraping.com/places/default/index'

## 6.1.1从浏览器加载cookie

In [ ]:
!sqlite3 D:/0ilraypan/git_jia/《用Python写网络爬虫2》/data/Cookies

In [3]:
from chp6.firefox_sessions import load_ff_sessions,find_ff_sessions
session_filename = find_ff_sessions()
cookies = load_ff_sessions(session_filename)
url = 'http://example.python-scraping.com'
html = requests.get(url, cookies=cookies)

TypeError: stat: path should be string, bytes, os.PathLike or integer, not NoneType

In [6]:
from lxml.html import fromstring
tree = fromstring(html.content)
tree.cssselect('ul#navbar li a')[0].text_content()

NameError: name 'html' is not defined